# Align
---

#### Overview
Interactive 3D alignment of serial sections.

In [15]:
import pathlib
import requests

from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import renderapi
import os

from scripted_render_pipeline import basic_auth
from scripted_render_pipeline.importer import uploader
from interactive_render import plotting
from interactive_render.utils import clear_image_cache

#### `render-ws` environment variables

In [16]:
# create an authorized session
auth = basic_auth.load_auth()
sesh = requests.Session()
sesh.auth = auth

# render-ws environment variables
params_render = {
    "host": "http://localhost",
    "port": 8081,
    "client_scripts": "/home/catmaid/render/render-ws-java-client/src/main/scripts",
    "client_script": "/home/catmaid/render/render-ws-java-client/src/main/scripts/run_ws_client.sh",
    "owner": "akievits",
    "project": "20230914_RP_exocrine_partial_test",
    "session": sesh
}

params_uploader = {
    "host": "https://sonic.tnw.tudelft.nl",
    "owner": "akievits",
    "project": "20230914_RP_exocrine_partial_test",
    "auth": auth
}

# set project directory
dir_project = pathlib.Path("/long_term_storage/akievits/FAST-EM/20230914_RP_exocrine_partial_test/")

## 2) Rough alignment (I)
---
Perform rough alignment of downsamples section images

### Create downsampled montage stack

In [17]:
from interactive_render.utils import create_downsampled_stack

In [18]:
# Set stacks for downsampling
stack_2_downsample = {
    'in': 'postcorrection_stitched',
    'out': 'postcorrection_dsmontages'
}

In [19]:
# Create downsampled stack
ds_stack = create_downsampled_stack(dir_project, 
                                    stack_2_downsample, 
                                    **params_render)
# Upload
# initialize uploader
uppity = uploader.Uploader(
        **params_uploader,
        clobber=False
)

# import stack to render-ws
uppity.upload_to_render(
    stacks=[ds_stack],
    z_resolution=100
)

  0%|          | 0/3 [00:00<?, ?it/s]

uploading: 100%|██████████| 1/1 [00:00<00:00,  2.09stacks/s]


### Inspect downsampled montage stack

In [20]:
# plot stack
plotting.plot_stacks(
    [ds_stack.name],
    width=1000,
    vmin=0,
    vmax=65535,
    **params_render
)

  0%|          | 0/1 [00:00<?, ?it/s]

interactive(children=(IntSlider(value=0, description='z', max=2), IntSlider(value=22338, description='vmin', m…

## 3) Rough alignment (II)
---
Get point matches for `dsmontage` stack and roughly align
### Align `dsmontage` stack


In [21]:
ds_stack_2_align = {
    'in': 'postcorrection_dsmontages',
    'out': 'postcorrection_dsmontages_aligned'
}

### Get point matches

Use `render-ws` `PointMatchClient` script to find matching features between the neighboring z-levels
#### Collect tile pairs

In [22]:
# choose stack from which to get tile pairs
z_values = [int(z) for z in renderapi.stack.get_z_values_for_stack(
    ds_stack_2_align['in'],
    **params_render
)]

# Get tile pairs from the rough aligned stack
tilepairs = renderapi.client.tilePairClient(
    stack=ds_stack_2_align['in'],
    minz=min(z_values),
    maxz=max(z_values),
    zNeighborDistance=1,  # half-height of search cylinder
    excludeSameLayerNeighbors=False,
    subprocess_mode="check_output",  # suppresses output
    **params_render
)["neighborPairs"]

# Show tile pairs
out = f"Number of tile pairs... {len(tilepairs)}"
print(out, "\n" + "-"*len(out))
tilepairs[:5]

Number of tile pairs... 2 
-------------------------


[{'p': {'groupId': 'S003', 'id': 't0_z0.0_y0_x0'},
  'q': {'groupId': 'S004', 'id': 't0_z1.0_y0_x0'}},
 {'p': {'groupId': 'S004', 'id': 't0_z1.0_y0_x0'},
  'q': {'groupId': 'S005', 'id': 't0_z2.0_y0_x0'}}]

In [23]:
from renderapi.client import (
    SiftPointMatchOptions,
    MatchDerivationParameters,
    FeatureExtractionParameters
)

In [24]:
# Name for pointmatch collection
match_collection = f"{params_render['project']}_{ds_stack_2_align['in']}_matches"
match_collection

'20230914_RP_exocrine_partial_test_postcorrection_dsmontages_matches'

In [25]:
# renderapi.pointmatch.delete_collection(match_collection,
#                                        **params_render)

#### Set SIFT + RANSAC parameters

In [32]:
# `RANSAC` parameters
params_RANSAC = MatchDerivationParameters(
    matchIterations=None,
    matchMaxEpsilon=25,        # maximal alignment error
    matchMaxNumInliers=None,
    matchMaxTrust=None,
    matchMinInlierRatio=0.05,  # minimal inlier ratio
    matchMinNumInliers=7,      # minimal number of inliers
    matchModelType='AFFINE',   # expected transformation
    matchRod=0.92              # closest/next closest ratio
)

# `SIFT` parameters
params_SIFT = FeatureExtractionParameters(
    SIFTfdSize=8,              # feature descriptor size
    SIFTmaxScale=0.50,         # (width/height *) maximum image size
    SIFTminScale=0.1,         # (width/height *) minimum image size
    SIFTsteps=5                # steps per scale octave
)

# Combined `SIFT` & `RANSAC` parameters
params_SIFT = SiftPointMatchOptions(
    fillWithNoise=True,
    **{**params_RANSAC.__dict__,
       **params_SIFT.__dict__}
)

# Extra parameters
params_SIFT.numberOfThreads = 1  # multithreading
params_SIFT.__dict__

{'SIFTfdSize': 8,
 'SIFTmaxScale': 0.5,
 'SIFTminScale': 0.1,
 'SIFTsteps': 7,
 'matchIterations': None,
 'matchMaxEpsilon': 25,
 'matchMaxNumInliers': None,
 'matchMaxTrust': None,
 'matchMinInlierRatio': 0.05,
 'matchMinNumInliers': 7,
 'matchModelType': 'AFFINE',
 'matchRod': 0.92,
 'renderScale': None,
 'fillWithNoise': True,
 'numberOfThreads': 1}

In [33]:
# Loop through tile pairs
for tp in tqdm(tilepairs):

    # Format tile pair
    tp_ids = (tp["p"]["id"], tp["q"]["id"])

    # Run SIFT + RANSAC via render-ws PointMatchClient
    renderapi.client.pointMatchClient(
        stack=ds_stack_2_align['in'],
        collection=match_collection,
        tile_pairs=[tp_ids],
        sift_options=params_SIFT,
        excludeAllTransforms=True,
        subprocess_mode='check_output',  # suppresses output
        **params_render
    )

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
plotting.plot_dsstack_with_alignment_matches(
    ds_stack_2_align['in'],
    match_collection,
    width=1000,
    **params_render
)

  0%|          | 0/1 [00:00<?, ?it/s]

interactive(children=(IntSlider(value=0, description='z', max=1), Output()), _dom_classes=('widget-interact',)…